# 1 Keypoints

Keypoints are certain locations in the image that are very distinct and could be identified between two or more different image.

Available methods to find locally distinct points in an image include:

- Harris corner detector
- Shi-Tomasi corner detector
- Föstner operator
- Difference of Gaussians

## Corners and edges

Corners and edges are often highly distinct points.

Corners are invariant to translation, rotation and illumination.
They normally have two orthogonal edges.

## Finding corners

To find corners we need to search for intensity changes in two directions.

Compute SSD (sum of squared differences) of neighbor pixels around (x,y).

$$
f(x,y)=\underbrace{\sum_{(u,v)\in W_{xy}}}_{\text{local patch around x,y}} (I(u,v)-I(u+\delta u, v+\delta v))^2
$$

Using Taylor series we obtain:

$$I(u+\delta u, v+\delta v) \approx I(u,v) + [J_x, J_y] \begin{bmatrix}\delta u \\ \delta v\end{bmatrix}$$

Putting this term in the equation above we get:

$$f(x,y)\approx \sum_{(u,v)\in W_{xy}} (I(u,v)-I(u,v) + [J_x, J_y] \begin{bmatrix}\delta u \\ \delta v\end{bmatrix})^2$$

$$f(x,y)\approx \sum_{(u,v)\in W_{xy}} ([J_x, J_y] \begin{bmatrix}\delta u \\ \delta v\end{bmatrix})^2$$

In a matrix form:

$$f(x,y) \approx \sum_{(u,v)\in W_{xy}} \Big(\begin{bmatrix}\delta u \\ \delta v\end{bmatrix}^T \begin{bmatrix} J_x^2 & J_xJ_y \\ J_xJ_y & J_y^2\end{bmatrix} \begin{bmatrix}\delta u \\ \delta v\end{bmatrix} \Big)^2$$

We can move the sum inside the matrix.

$$f(x,y) \approx \Big(\begin{bmatrix}\delta u \\ \delta v\end{bmatrix}^T
\underbrace{
\begin{bmatrix} \sum_W J_x^2 & \sum_W J_xJ_y \\
\sum_W J_xJ_y &\sum_W J_y^2
\end{bmatrix}}_{\text{structure matrix}}
\begin{bmatrix}\delta u \\ \delta v\end{bmatrix} \Big)^2$$

Structure matrix summarizes the first derivatives of the image in a local patch and accumulates gradience.

$$M = \begin{bmatrix} \sum_W J_x^2 & \sum_W J_xJ_y \\
\sum_W J_xJ_y &\sum_W J_y^2
\end{bmatrix}$$

If we compute eigenvalues and eigenvectors of this matrix.
So it encodes the intensity changes in a local patch. It's just built from image gradience.

The Jacobians can be computed by convolving image either with Sharr or Sobel kernels:

$$
\begin{split}
J_x^2 &= (D_x * I)^2 \\
J_xJ_Y &= (D_x * I)(D_y * I) \\
J_y^2 &= (D_y * I)^2
\end{split}
$$

## Practical considerations

- RGB should be converted to gray-scale first.
- Real images are affected by noise, so smoothing of the input is suggested.

![corner detection pipeline](doc/corner_detection_pipeline.png)